#### This imitates a daily report on potentially dangerous earthquakes. This reports averages last 10 days of magnitude value and indicates places where average last 10 days magnitude is above 4.

In [289]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import math
from datetime import datetime as dt
from IPython.display import Image, HTML



from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from arcgis.features.analysis import interpolate_points

In [290]:

gis = GIS()
england_border = gis.content.search('england1 owner:api_data_owner', 'feature layer')
england_border

[]

In [291]:

import arcgis
from arcgis.gis import GIS

ago_gis = GIS() # Connect to ArcGIS Online as an anonymous user
search_subset = ago_gis.content.search("Earthquakes in Canada 2010-2019", item_type = "Feature Layer")
subset_item = search_subset[0]
subset_item

<Item title:"Find_Earthquakes_in_Canada_2010_2019___Earthquakes_2010_2020" type:Feature Layer Collection owner:wang1262_utoronto>

In [292]:
canada_map = gis.map('Canada')
canada_map.add_layer(subset_item)
canada_map

MapView(layout=Layout(height='400px', width='100%'))

In [293]:
from dateutil.relativedelta import relativedelta
import datetime

subset_feature_layer = subset_item.layers[0]

query_result = subset_feature_layer.query()


att_data_frame = query_result.sdf # get as a Pandas dataframe
att_data_frame

2018-05-07 13:30:57.796555


,OBJECTID,magnitude_codelist,magnitude,magnitude_type,date,place,depth,latitude,longitude,SHAPE
0,1,<2,1.7,ML,2010-01-01T00:16:49+0000,81 km NE of Seattle,0.000000,48.192001,-121.677002,"{""x"": -1931959.1460000016, ""y"": 321163.0564999..."
1,2,<2,1.6,ML,2010-01-01T04:15:17+0000,83 km W of Gold R.,11.600000,49.500999,-127.222000,"{""x"": -2231203.6358000003, ""y"": 633188.6411000..."
2,3,<2,1.7,ML,2010-01-01T08:51:55+0000,59 km SE of Victoria,21.700001,47.980999,-122.935997,"{""x"": -2027295.0557000004, ""y"": 338922.4587999..."
3,4,2,2.3,ML,2010-01-01T12:24:40+0000,139 km SSW of Sandspit,20.000000,52.028000,-132.257004,"{""x"": -2387745.772, ""y"": 1055017.2118999995, ""..."
4,5,2,2.5,ML,2010-01-01T12:49:52+0000,130 km SSE of Sandspit,20.000000,52.125000,-131.266006,"{""x"": -2325637.0227999985, ""y"": 1027226.690499..."
...,...,...,...,...,...,...,...,...,...,...
22406,22407,2,2.0,ML,2019-12-30T21:00:26+0000,"30 km SE of Ashcroft, BC",0.000000,50.485001,-121.069000,"{""x"": -1788910.4444000013, ""y"": 536527.3531999..."
22407,22408,3,3.4,Mw,2019-12-31T12:22:46+0000,"217 km W of Port Hardy, BC",10.000000,50.912998,-130.570999,"{""x"": -2356714.0801, ""y"": 888723.8988000005, ""..."
22408,22409,2,2.2,ML,2019-12-31T17:13:50+0000,"24 km E of Sidney, BC",13.800000,48.588001,-123.097000,"{""x"": -2009285.4041000009, ""y"": 405287.8839999..."
22409,22410,<2,1.8,ML,2019-12-31T18:37:24+0000,"31 km SE of Ashcroft, BC",14.000000,50.487999,-121.035004,"{""x"": -1786629.1176999994, ""y"": 535862.5709999..."


In [294]:
att_data_frame['running_avg'] = att_data_frame.sort_values(by=['date'], ascending=True)\
                       .groupby(['place'])['magnitude']\
                       .rolling(10, min_periods = 1).mean()\
                       .reset_index(drop=True, level=0)

In [295]:
mx =att_data_frame['date'].max()
rslt_df = att_data_frame.loc[(att_data_frame['date'] == mx)&(att_data_frame['running_avg'] > 4)]
rslt_df

,OBJECTID,magnitude_codelist,magnitude,magnitude_type,date,place,depth,latitude,longitude,SHAPE,running_avg
